In [1]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT" -O dart.csv && rm -rf /tmp/cookies.txt

In [5]:
from gensim.models import doc2vec
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
from datasets import load_dataset

df = pd.read_csv('dart_v2_1.csv') # loda dataset으로 똑같은거 불러오시면 됩니다.


In [6]:
df

,Unnamed: 0,stock_code,report_idx,corp_code,corp_name,report_date,clean,회사의 개요,사업의 개요
0,0,950130,20211115000151,956028,엑세스바이오,20170630,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 ...,"1. 사업의 개요당사는 체외진단 기술을 토대로 면역화학진단, 바이오센서, 분자진단..."
1,1,900070,20211125000049,783246,글로벌에스엠,20170703,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 ...,"1. 사업의 개요지주회사(持株會社, Holding Company)란 다른 회사의 주..."
2,2,900120,20211028000448,800084,씨케이에이치,20170703,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요 (1) 연결대상 종속회사 개황연결대상 종속회사 현황(요약) (단위...,1. 사업의 개요가. 산업개황 및 전망 당사의 매출과 이익은 중국 내에서의 건강식품...
3,3,900290,20211028000456,1170962,GRT,20181122,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요가. 연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결대...,1. 사업의 개요가. 산업의 이해 (1) 영위하고 있는 사업당사의 주요 사업영역으...
4,4,900340,20210830000685,1328639,윙입푸드,20181130,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요가. 연결대상 종속회사 개황 1-1 연결대상 종속회사 현황(요약)...,1. 사업의 개요 당사는 한국 표준산업 분류 기준 대분류상 식료품 제조업 산업에...
...,...,...,...,...,...,...,...,...,...
2308,2308,17800,20211115002172,164724,현대엘리베이터,20210512,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요가. 회사의 법적ㆍ상업적 명칭 - 당사의 명칭은 '현대엘리베이...,"1. 사업의 개요◆ 당사는 1984년 5월 23일 설립하여 엘리베이터, 에스컬레이터..."
2309,2309,3410,20211115002437,138224,쌍용씨앤이,20211020,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요 가. 연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결...,"1. 사업의 개요 쌍용C&E㈜는 시멘트사업을 중심으로 환경자원사업, 레미콘사업, ..."
2310,2310,15760,20211115001967,159193,한국전력공사,20210602,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요 한국전력공사는 1982년 1월 1일 설립되어 1989년 8월 1...,"1. 사업의 개요현재 국내 전력 산업의 체계는 전력생산, 수송, 판매 체계로 이뤄지..."
2311,2311,900140,20211126000686,838500,엘브이엠씨,20210611,"{'【 대표이사 등의 확인 】': {}, 'I. 회사의 개요': {'1. 회사의 개...",1. 회사의 개요가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 ...,1. 사업의 개요당사는 2009년 6월 16일 라오스 소재의 사업자회사인 Auto ...


In [7]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['clean']
  tag = row['corp_name']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

100%|██████████| 2313/2313 [07:13<00:00,  5.34it/s]

문서의 수 : 2313


In [39]:
# tagged_corpus_list[0]


In [6]:
from gensim.models import doc2vec
import gensim

# !pip install gensim==3.8.3
gensim.__version__

'3.8.3'

In [24]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

Tag Size: 2313 / 

In [38]:
model = doc2vec.Doc2Vec.load('dart.doc2vec')
similar_doc = model.docvecs.most_similar('대상', topn=10)
print(similar_doc)

[('대상홀딩스', 0.6068530082702637), ('신송홀딩스', 0.4938071072101593), ('샘표식품', 0.4451195001602173), ('CJ제일제당', 0.43365150690078735), ('동원F&B', 0.4312994182109833), ('네오크레마', 0.42146843671798706), ('CJ씨푸드', 0.42137080430984497), ('대한제당', 0.42110705375671387), ('샘표', 0.4198298454284668), ('오뚜기', 0.41936951875686646)]


In [29]:
model = doc2vec.Doc2Vec.load('dart100.doc2vec')
similar_doc = model.docvecs.most_similar('삼성전자', topn=10)
print(similar_doc)

[('삼성전기', 0.5503195524215698), ('코미코', 0.45464521646499634), ('LG디스플레이', 0.4512143135070801), ('솔루엠', 0.43255314230918884), ('LG전자', 0.4290298521518707), ('현대모비스', 0.41662076115608215), ('삼성SDI', 0.41464823484420776), ('영풍', 0.41451069712638855), ('피에스케이', 0.40894365310668945), ('화신', 0.4028289318084717), ('신성이엔지', 0.40143483877182007), ('원익IPS', 0.40046101808547974), ('AP시스템', 0.39835667610168457), ('LG화학', 0.3982613682746887), ('심텍홀딩스', 0.39380714297294617), ('현대자동차', 0.3906320631504059), ('이라이콤', 0.3900991380214691), ('에이스테크', 0.38972240686416626), ('이마트', 0.3890407979488373), ('한미반도체', 0.38878360390663147)]
